In [1]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe
tf.enable_eager_execution()

import numpy as np
import time
import random
import string
import pandas as pd
import copy
import os

os.chdir('/home/scott/Documents/EDx/MIT/Python/Spyder_scripts/')
WORDLIST_FILENAME = "words.txt"


In [2]:
def loadWords():
    """
    Returns a list of valid words. Words are strings of lowercase letters.
    
    Depending on the size of the word list, this function may
    take a while to finish.
    """
    print("Loading word list from file...")
    # inFile: file
    inFile = open(WORDLIST_FILENAME, 'r')
    # line: string
    line = inFile.readline()
    # wordlist: list of strings
    wordlist = line.split()
    print("  ", len(wordlist), "words loaded.")
    wordlist = tf.convert_to_tensor(wordlist)
    return wordlist

def chooseWord(wordlist):
    """
    wordlist (list): list of words (strings)

    Returns a word from wordlist at random
    """
    return np.random.choice(wordlist)

def isWordGuessed(secretWord, lettersGuessed):
    '''
    secretWord: string, the word the user is guessing
    lettersGuessed: list, what letters have been guessed so far
    returns: boolean, True if all the letters of secretWord are in lettersGuessed;
      False otherwise
    '''
    for char in secretWord:
        if char not in lettersGuessed:
            return False
    
    return True  


def getGuessedWord(secretWord, lettersGuessed):
    '''
    secretWord: string, the word the user is guessing
    lettersGuessed: list, what letters have been guessed so far
    returns: string, comprised of letters and underscores that represents
      what letters in secretWord have been guessed so far.
    '''
    
    word = str(secretWord)
    
    for char in word:
        if char not in lettersGuessed:
            word = word.replace(char,'_')
            
    return word
    


def getAvailableLetters(lettersGuessed):
    '''
    lettersGuessed: list, what letters have been guessed so far
    returns: string, comprised of letters that represents what letters have not
      yet been guessed.
    '''
    
    availableLetters = 'abcdefghijklmnopqrstuvwxyz'
    for char in lettersGuessed:
            position = availableLetters.index(char)
            availableLetters = availableLetters[0:position] + availableLetters[position+1::]
            
    return availableLetters

def Secrets():
    
    inFile = open(WORDLIST_FILENAME, 'r')
    # line: string
    line = inFile.readline()
    # wordlist: list of strings
    wordlist = line.split()
    # Construct pandas DataFrame called secrets that sorts all words in wordlist according to word length
    columns = ['1','2','3','4','5','6','7','8','9','10','11','12','13']
    secrets = pd.DataFrame()
    
    for i in range(1,max(len(word) for word in wordlist)+1):
        dat = []
        for word in wordlist:
            if len(word) == i:
                dat.append(word)
        dat = pd.Series(dat)
        secrets = pd.concat([secrets,dat],axis=1, join='outer', join_axes=None, ignore_index=False)
    secrets.columns = columns
    
    return secrets

In [3]:
# This function creates an empty DataFrame like object that is a Tensor 
# This object can be trained

def createLife():
    inFile = open(WORDLIST_FILENAME, 'r')
    # line: string
    line = inFile.readline()
    # wordlist: list of strings
    words = line.split()
    columns = list(range(1,max(len(word) for word in words)+1))
    index = list(string.ascii_lowercase)
    brain = pd.DataFrame(index = index, columns = columns)
    for i in range(1,14):
        brain[i] = 0
    return tf.convert_to_tensor(brain.values)





In [6]:
def tensorPlayerAI(brain,numberOfGames):
    
    start = time.time()
    wordlist = loadWords()
    n = 1
    gamesWon = 0
    letters = list(string.ascii_lowercase)
    neuron = tf.convert_to_tensor(brain, dtype=None, name='titans', preferred_dtype=None)
    rawLetters = string.ascii_lowercase
    secrets = Secrets()
    
    while n <= numberOfGames:
        
        secretWord = np.random.choice(wordlist).decode("utf-8")
        lettersGuessed = ''
        numberOfGuesses = 8
        col = str(len(secretWord))
        words = secrets[col].dropna()
        words = list(words)
        print('Welcome to the game Hangman!')
        print('I am thinking of a word that is ' + str(len(secretWord)) + ' letters long.')
        print('-------------')
        game = True
        
        while game == True:
            
            options = copy.deepcopy(words)
            choices = secrets[str(len(secretWord))]
            availableLetters = getAvailableLetters(lettersGuessed)
            print('You have ' + str(numberOfGuesses) + ' guesses left.')
            print('Available letters: ' + availableLetters)
            best = 0
            
            if len(availableLetters) < 26:
                for chance in words:
                    for j in range(0,len(chance)):
                        if word[j] != '_' and  word[j] != chance[j]:
                            options.remove(chance)
                            break     
                           
            for letter in availableLetters:
        
                maximum = 0
                for word in options:
                    for char in word:
                        if char == letter:
                            maximum += 1
                            
                if maximum > best:
                    best = maximum
                    choice = letter
                    letterGuessed = choice
                    
            lettersGuessed = lettersGuessed[0::] + letterGuessed
            word = getGuessedWord(secretWord, lettersGuessed)
            letter = lettersGuessed[-1]  
            
            if lettersGuessed[-1] not in secretWord:
                print("Oops! That letter is not in my word: " + word)
                print('-------------')
                numberOfGuesses -= 1
                   
            else:
                print('Good guess: ' + word)
                print('-------------')
                for i in range(0,26):
                    if letters[i] == letterGuessed:
                        AI = neuron.numpy()
                        AI[i,len(secretWord)-1] += 1
                        neuron = tf.convert_to_tensor(AI, dtype=None, name='titans', preferred_dtype=None)

            answer = isWordGuessed(secretWord,lettersGuessed)
        
            if numberOfGuesses == 0 or answer == True:
                game = False    
           
        if answer == True:
            print('Congratulations, you won!')
            gamesWon += 1
            n += 1
            
        else:
            print('Sorry, you ran out of guesses. The word was ' + secretWord)
            n += 1
            
    print('Player function won ' + str(gamesWon) + ' games.')
    print('Player function lost ' + str(numberOfGames - gamesWon) + ' games.')
    end = time.time()
    print('PlayerAI played ' + str(numberOfGames) + ' games in '+ str(round((end-start)/60,2)) + ' minutes')
    
    return [numberOfGames,neuron]
        

In [7]:
numberOfGames = 100
babyBrain = createLife()
[n,brain] = tensorPlayerAI(babyBrain,numberOfGames)


Loading word list from file...
   55909 words loaded.
Welcome to the game Hangman!
I am thinking of a word that is 6 letters long.
-------------
You have 8 guesses left.
Available letters: abcdefghijklmnopqrstuvwxyz
Good guess: _____e
-------------
You have 8 guesses left.
Available letters: abcdfghijklmnopqrstuvwxyz
Good guess: _a_a_e
-------------
You have 8 guesses left.
Available letters: bcdfghijklmnopqrstuvwxyz
Oops! That letter is not in my word: _a_a_e
-------------
You have 7 guesses left.
Available letters: bcdfhijklmnopqrstuvwxyz
Good guess: _a_ace
-------------
You have 7 guesses left.
Available letters: bdfhijklmnopqrstuvwxyz
Good guess: _alace
-------------
You have 7 guesses left.
Available letters: bdfhijkmnopqrstuvwxyz
Good guess: palace
-------------
Congratulations, you won!
Welcome to the game Hangman!
I am thinking of a word that is 10 letters long.
-------------
You have 8 guesses left.
Available letters: abcdefghijklmnopqrstuvwxyz
Good guess: ______e___
---------

Good guess: _____a_le
-------------
You have 8 guesses left.
Available letters: bcdfghijkmnopqrstuvwxyz
Good guess: _____able
-------------
You have 8 guesses left.
Available letters: cdfghijkmnopqrstuvwxyz
Oops! That letter is not in my word: _____able
-------------
You have 7 guesses left.
Available letters: cdfghijkmnopqstuvwxyz
Good guess: i____able
-------------
You have 7 guesses left.
Available letters: cdfghjkmnopqstuvwxyz
Oops! That letter is not in my word: i____able
-------------
You have 6 guesses left.
Available letters: cdfghjkmopqstuvwxyz
Good guess: imm__able
-------------
You have 6 guesses left.
Available letters: cdfghjkopqstuvwxyz
Good guess: immo_able
-------------
You have 6 guesses left.
Available letters: cdfghjkpqstuvwxyz
Good guess: immovable
-------------
Congratulations, you won!
Welcome to the game Hangman!
I am thinking of a word that is 10 letters long.
-------------
You have 8 guesses left.
Available letters: abcdefghijklmnopqrstuvwxyz
Good guess: ______

Good guess: ____e__
-------------
You have 8 guesses left.
Available letters: abcdfghijklmnopqrstuvwxyz
Oops! That letter is not in my word: ____e__
-------------
You have 7 guesses left.
Available letters: abcdfghijklmnopqrtuvwxyz
Good guess: ____er_
-------------
You have 7 guesses left.
Available letters: abcdfghijklmnopqtuvwxyz
Oops! That letter is not in my word: ____er_
-------------
You have 6 guesses left.
Available letters: abcdfghijklmnopquvwxyz
Good guess: _a__er_
-------------
You have 6 guesses left.
Available letters: bcdfghijklmnopquvwxyz
Oops! That letter is not in my word: _a__er_
-------------
You have 5 guesses left.
Available letters: bcdfghijklmopquvwxyz
Good guess: _al_er_
-------------
You have 5 guesses left.
Available letters: bcdfghijkmopquvwxyz
Good guess: hal_er_
-------------
You have 5 guesses left.
Available letters: bcdfgijkmopquvwxyz
Good guess: halber_
-------------
You have 5 guesses left.
Available letters: cdfgijkmopquvwxyz
Good guess: halberd
-----

Good guess: ____i_ate
-------------
You have 8 guesses left.
Available letters: bcdfghjklmnopqrsuvwxyz
Oops! That letter is not in my word: ____i_ate
-------------
You have 7 guesses left.
Available letters: bdfghjklmnopqrsuvwxyz
Good guess: ____inate
-------------
You have 7 guesses left.
Available letters: bdfghjklmopqrsuvwxyz
Oops! That letter is not in my word: ____inate
-------------
You have 6 guesses left.
Available letters: bdfghjklopqrsuvwxyz
Oops! That letter is not in my word: ____inate
-------------
You have 5 guesses left.
Available letters: dfghjklopqrsuvwxyz
Good guess: __llinate
-------------
You have 5 guesses left.
Available letters: dfghjkopqrsuvwxyz
Good guess: _ollinate
-------------
You have 5 guesses left.
Available letters: dfghjkpqrsuvwxyz
Good guess: pollinate
-------------
Congratulations, you won!
Welcome to the game Hangman!
I am thinking of a word that is 7 letters long.
-------------
You have 8 guesses left.
Available letters: abcdefghijklmnopqrstuvwxyz
O

Good guess: trepans
-------------
Congratulations, you won!
Welcome to the game Hangman!
I am thinking of a word that is 6 letters long.
-------------
You have 8 guesses left.
Available letters: abcdefghijklmnopqrstuvwxyz
Oops! That letter is not in my word: ______
-------------
You have 7 guesses left.
Available letters: abcdfghijklmnopqrstuvwxyz
Good guess: _____s
-------------
You have 7 guesses left.
Available letters: abcdfghijklmnopqrtuvwxyz
Oops! That letter is not in my word: _____s
-------------
You have 6 guesses left.
Available letters: bcdfghijklmnopqrtuvwxyz
Good guess: _r___s
-------------
You have 6 guesses left.
Available letters: bcdfghijklmnopqtuvwxyz
Oops! That letter is not in my word: _r___s
-------------
You have 5 guesses left.
Available letters: bcdfghijklmnopquvwxyz
Oops! That letter is not in my word: _r___s
-------------
You have 4 guesses left.
Available letters: bcdfghijklmnpquvwxyz
Oops! That letter is not in my word: _r___s
-------------
You have 3 guesse

...

Oops! That letter is not in my word: e_cl__e
-------------
You have 3 guesses left.
Available letters: bdfghjkmopqrsuwxyz
Good guess: excl__e
-------------
You have 3 guesses left.
Available letters: bdfghjkmopqrsuwyz
Good guess: excl_de
-------------
You have 3 guesses left.
Available letters: bfghjkmopqrsuwyz
Good guess: exclude
-------------
Congratulations, you won!
Welcome to the game Hangman!
I am thinking of a word that is 9 letters long.
-------------
You have 8 guesses left.
Available letters: abcdefghijklmnopqrstuvwxyz
Good guess: ___e___e_
-------------
You have 8 guesses left.
Available letters: abcdfghijklmnopqrstuvwxyz
Good guess: ___e___es
-------------
You have 8 guesses left.
Available letters: abcdfghijklmnopqrtuvwxyz
Oops! That letter is not in my word: ___e___es
-------------
You have 7 guesses left.
Available letters: abcdfghjklmnopqrtuvwxyz
Good guess: ___e_a_es
-------------
You have 7 guesses left.
Available letters: bcdfghjklmnopqrtuvwxyz
Good guess: ___e_ates


Welcome to the game Hangman!
I am thinking of a word that is 7 letters long.
-------------
You have 8 guesses left.
Available letters: abcdefghijklmnopqrstuvwxyz
Good guess: _____e_
-------------
You have 8 guesses left.
Available letters: abcdfghijklmnopqrstuvwxyz
Oops! That letter is not in my word: _____e_
-------------
You have 7 guesses left.
Available letters: abcdfghijklmnopqrtuvwxyz
Good guess: _____ed
-------------
You have 7 guesses left.
Available letters: abcfghijklmnopqrtuvwxyz
Oops! That letter is not in my word: _____ed
-------------
You have 6 guesses left.
Available letters: abcfghijkmnopqrtuvwxyz
Good guess: _r___ed
-------------
You have 6 guesses left.
Available letters: abcfghijkmnopqtuvwxyz
Good guess: _ro__ed
-------------
You have 6 guesses left.
Available letters: abcfghijkmnpqtuvwxyz
Oops! That letter is not in my word: _ro__ed
-------------
You have 5 guesses left.
Available letters: abcfghijkmnqtuvwxyz
Good guess: gro__ed
-------------
You have 5 guesses lef

Good guess: __mbled
-------------
You have 6 guesses left.
Available letters: cfghijknopqrtuvwxyz
Good guess: _umbled
-------------
You have 6 guesses left.
Available letters: cfghijknopqrtvwxyz
Oops! That letter is not in my word: _umbled
-------------
You have 5 guesses left.
Available letters: cghijknopqrtvwxyz
Oops! That letter is not in my word: _umbled
-------------
You have 4 guesses left.
Available letters: cgijknopqrtvwxyz
Oops! That letter is not in my word: _umbled
-------------
You have 3 guesses left.
Available letters: cgiknopqrtvwxyz
Good guess: rumbled
-------------
Congratulations, you won!
Welcome to the game Hangman!
I am thinking of a word that is 5 letters long.
-------------
You have 8 guesses left.
Available letters: abcdefghijklmnopqrstuvwxyz
Oops! That letter is not in my word: _____
-------------
You have 7 guesses left.
Available letters: abcdefghijklmnopqrtuvwxyz
Good guess: __e__
-------------
You have 7 guesses left.
Available letters: abcdfghijklmnopqrtuv

Oops! That letter is not in my word: ________
-------------
You have 6 guesses left.
Available letters: abcdfghijklmnopqrtuvwxyz
Good guess: _____i__
-------------
You have 6 guesses left.
Available letters: abcdfghjklmnopqrtuvwxyz
Good guess: ___n_in_
-------------
You have 6 guesses left.
Available letters: abcdfghjklmopqrtuvwxyz
Oops! That letter is not in my word: ___n_in_
-------------
You have 5 guesses left.
Available letters: abcdfhjklmopqrtuvwxyz
Oops! That letter is not in my word: ___n_in_
-------------
You have 4 guesses left.
Available letters: abcdfhjklmopqruvwxyz
Oops! That letter is not in my word: ___n_in_
-------------
You have 3 guesses left.
Available letters: bcdfhjklmopqruvwxyz
Oops! That letter is not in my word: ___n_in_
-------------
You have 2 guesses left.
Available letters: bcdfhjklmopqrvwxyz
Oops! That letter is not in my word: ___n_in_
-------------
You have 1 guesses left.
Available letters: bcdfhjkmopqrvwxyz
Oops! That letter is not in my word: ___n_in_


Oops! That letter is not in my word: _____e_
-------------
You have 7 guesses left.
Available letters: abcdfghijklmnopqrtuvwxyz
Oops! That letter is not in my word: _____e_
-------------
You have 6 guesses left.
Available letters: abcfghijklmnopqrtuvwxyz
Good guess: _____er
-------------
You have 6 guesses left.
Available letters: abcfghijklmnopqtuvwxyz
Oops! That letter is not in my word: _____er
-------------
You have 5 guesses left.
Available letters: abcfghjklmnopqtuvwxyz
Oops! That letter is not in my word: _____er
-------------
You have 4 guesses left.
Available letters: abcfghjklmnopquvwxyz
Oops! That letter is not in my word: _____er
-------------
You have 3 guesses left.
Available letters: bcfghjklmnopquvwxyz
Good guess: _l___er
-------------
You have 3 guesses left.
Available letters: bcfghjkmnopquvwxyz
Good guess: bl___er
-------------
You have 3 guesses left.
Available letters: cfghjkmnopquvwxyz
Oops! That letter is not in my word: bl___er
-------------
You have 2 guesses l

In [8]:
# Save the trained brain as a .npy file
# .npy files are saved in binary format
os.chdir('/home/scott/Documents/EDx/MIT/Python/Jupyter Notebooks/')
np.save('brain',brain.numpy())

In [9]:
# Shows how to load a .npy file
trainedBrain = np.load('brain.npy')

In [11]:
# Shows how to create a tensor variable from a numpy array
trainedBrain = tf.convert_to_tensor(trainedBrain, dtype=None, name='titans', preferred_dtype=None)
type(trainedBrain)
trainedBrain

<tf.Tensor: id=2711, shape=(26, 13), dtype=int64, numpy=
array([[ 0,  0,  1,  2,  5,  4, 11,  5, 15,  8,  0,  0,  0],
       [ 0,  0,  0,  0,  3,  0,  4,  1,  5,  4,  0,  0,  0],
       [ 0,  0,  0,  0,  1,  1,  3,  5,  7, 10,  0,  0,  0],
       [ 0,  0,  1,  0,  5,  2,  6,  3,  5, 11,  0,  0,  0],
       [ 0,  0,  1,  1,  7,  5, 14,  9, 17, 12,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  1,  1,  3,  2,  1,  0,  0,  0],
       [ 0,  0,  0,  0,  2,  2,  4,  0,  8,  2,  0,  0,  0],
       [ 0,  1,  0,  0,  4,  2,  4,  1,  3,  4,  0,  0,  0],
       [ 0,  0,  0,  0,  5,  2,  8,  7, 14,  5,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0],
       [ 0,  0,  0,  1,  0,  0,  3,  0,  0,  1,  0,  0,  0],
       [ 0,  0,  0,  0,  6,  2, 11,  6, 10,  2,  0,  0,  0],
       [ 0,  0,  0,  1,  1,  2,  1,  2,  6,  2,  0,  0,  0],
       [ 0,  0,  0,  0,  5,  3,  8,  6, 11,  5,  0,  0,  0],
       [ 0,  1,  0,  1,  7,  1,  4,  7, 11,  6,  0,  0,  0],
       [ 0,  0,  0,  0,  3, 

In [12]:
# I think its important to note my method of training is very basic.
# The algorithm used to train this brain will yield better results than implementing the brain for letter choice alone.
# Next step is to figure out how to train a ANN to play hangman. 
# A team at Microsoft azure has already achieved this using CTNK not tensorflow
# more information can be found at their github repo
# https://github.com/Azure/Hangman/blob/master/Train%20a%20Neural%20Network%20to%20Play%20Hangman.ipynb
